In [1]:
import sqlite3
import requests
from bs4 import BeautifulSoup

In [2]:
#Create the database
connection = sqlite3.connect('books_rate.db')
cursor = connection.cursor()

In [3]:
#1- create books & categories table
connection.execute('CREATE TABLE books (title VARCHAR(200), price NUMERIC, rate NUMERIC, book_catid INT)')
connection.execute('CREATE TABLE categories (category_id INT AUTO INCREAMENT, category_name VARCHAR(30))')

In [4]:
categories = []

In [5]:
#Retrieve categories and enumerate them
url = 'http://books.toscrape.com/'
response = requests.get(url)
soup = BeautifulSoup(response.text,'html.parser')

categories = []

retrieved = soup.find('div', attrs = {'class':'side_categories'}).find('ul').find_all('li')

for i in range(1,len(retrieved)):
    categories.append(retrieved[i].get_text().strip().lower())
    
#categories = categories[1:]

In [6]:
print(categories)

['travel', 'mystery', 'historical fiction', 'sequential art', 'classics', 'philosophy', 'romance', 'womens fiction', 'fiction', 'childrens', 'religion', 'nonfiction', 'music', 'default', 'science fiction', 'sports and games', 'add a comment', 'fantasy', 'new adult', 'young adult', 'science', 'poetry', 'paranormal', 'art', 'psychology', 'autobiography', 'parenting', 'adult fiction', 'humor', 'horror', 'history', 'food and drink', 'christian fiction', 'business', 'biography', 'thriller', 'contemporary', 'spirituality', 'academic', 'self help', 'historical', 'christian', 'suspense', 'short stories', 'novels', 'health', 'politics', 'cultural', 'erotica', 'crime']


In [7]:
#Insert into the categories table
for index, name in enumerate(categories):
    cursor.execute('INSERT INTO categories (category_id, category_name) VALUES(?,?)',[index,name])
    connection.commit()
connection.close()

In [8]:
print(cursor.lastrowid)

50


In [9]:
#Function that returns category id
def getCatID(name):
    for el in enumerate(categories):
        if name == el[1]: return el[0]

In [10]:
#Testing function
temp_id = getCatID('crime')
temp_id

49

In [11]:
#Function that returns rate in numerals
def getRate(rate):
    if rate == 'One': return 1
    elif rate == 'Two': return 2
    elif rate == 'Three': return 3
    elif rate == 'Four': return 4
    elif rate == 'Five': return 5

In [12]:
#Function that gets the changing part of link
def getTempWord(cat_name,temp_len,temp_id):
    temp_word = ''
    if temp_len == 1: return cat_name+'_'+str(getCatID(cat_name) + 2)
    else:
        for i in range(temp_len):
            split_name = cat_name.split()
            if i < temp_len - 1:
                temp_word += split_name[i] + '-'
            elif i == temp_len - 1:
                temp_word += split_name[i]
        temp_word +='_{}'.format(str(temp_id + 2))
        return temp_word

In [13]:
#Test the function
each_cat = dict()
for cat in categories:
    temp_id = getCatID(cat)
    temp_len = len(cat.split())
    temp_word = getTempWord(cat,temp_len,temp_id)
    print(temp_word)

travel_2
mystery_3
historical-fiction_4
sequential-art_5
classics_6
philosophy_7
romance_8
womens-fiction_9
fiction_10
childrens_11
religion_12
nonfiction_13
music_14
default_15
science-fiction_16
sports-and-games_17
add-a-comment_18
fantasy_19
new-adult_20
young-adult_21
science_22
poetry_23
paranormal_24
art_25
psychology_26
autobiography_27
parenting_28
adult-fiction_29
humor_30
horror_31
history_32
food-and-drink_33
christian-fiction_34
business_35
biography_36
thriller_37
contemporary_38
spirituality_39
academic_40
self-help_41
historical_42
christian_43
suspense_44
short-stories_45
novels_46
health_47
politics_48
cultural_49
erotica_50
crime_51


In [14]:
#Make a dictionary that has each category's books
each_cat = dict()
for cat in categories:
    temp_id = getCatID(cat)
    temp_len = len(cat.split())
    temp_word = getTempWord(cat,temp_len,temp_id)
    
    #Continue to read more pages
    more_pages = True
    
    #Mark page index
    page_index = 1
    
    #Inializing element of dictionary by an empty list
    each_cat[cat] = []
    
    while(more_pages):
        if page_index == 1:
            cat_url = 'http://books.toscrape.com/catalogue/category/books/{}/index.html'.format(temp_word)
            #print(cat_url)
        else:
            cat_url = 'http://books.toscrape.com/catalogue/category/books/{}/page-{}.html'.format(temp_word,page_index)
            
        response = requests.get(cat_url)
        soup = BeautifulSoup(response.text,'html.parser')
        
        #loop by a maximum of 20 books per page (possible to be less)
        for i in range(20):
            try:
                title = soup.find_all('h3')[i].find('a').attrs['title']
                each_cat[cat].append(title)
            except:
                more_pages = False
                break
            
        page_index += 1
        #         if title = None: 
           

In [15]:
#print the created dictionary containing each category and corresponding books
print(each_cat)

{'travel': ["It's Only the Himalayas", 'Full Moon over Noahâ\x80\x99s Ark: An Odyssey to Mount Ararat and Beyond', 'See America: A Celebration of Our National Parks & Treasured Sites', 'Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel', 'Under the Tuscan Sun', 'A Summer In Europe', 'The Great Railway Bazaar', 'A Year in Provence (Provence #1)', 'The Road to Little Dribbling: Adventures of an American in Britain (Notes From a Small Island #2)', 'Neither Here nor There: Travels in Europe', '1,000 Places to See Before You Die'], 'mystery': ['Sharp Objects', 'In a Dark, Dark Wood', 'The Past Never Ends', 'A Murder in Time', 'The Murder of Roger Ackroyd (Hercule Poirot #4)', 'The Last Mile (Amos Decker #2)', 'That Darkness (Gardiner and Renner #1)', 'Tastes Like Fear (DI Marnie Rome #3)', 'A Time of Torment (Charlie Parker #14)', 'A Study in Scarlet (Sherlock Holmes #1)', 'Poisonous (Max Revere Novels #3)', 'Murder at the 42nd Street Library (Raymond Ambler #1)', 'Most Wa

In [16]:
#Get the category of a book based on scrapping each category separately
def getBookCat(book_name):
    for key in each_cat:
        if book_name in each_cat[key]: return key

In [17]:
getBookCat('A Light in the Attic')

'poetry'

In [18]:
getBookCat('Tipping the Velvet')

'historical fiction'

In [19]:
url = 'http://books.toscrape.com/catalogue/page-{}.html'

In [20]:
for page in range(1,51):
    #Initiating url for every page
    finalurl = url.format(page)
    response = requests.get(finalurl)
    soup = BeautifulSoup(response.text,'html.parser')
    
    #get all prices in page
    prices = soup.find_all('p',attrs={'class':'price_color'})
    
    #get all ratings in page   
    temp_rate = soup.find_all('p', attrs={'class':'star-rating'})

    for book_num in range(0,20):
        #get title, price, rate
        try:
            title = soup.find_all('h3')[book_num].find('a').attrs['title']
            price = float(prices[book_num].get_text().strip('Â£'))
            rating = temp_rate[book_num].attrs['class'][1]
            #get rating in numeric value
            rating = getRate(rating)
            #get category of the book
            cat_name = getBookCat(title)
            print(f'{title} in category {cat_name} was inserted successfully!')
            #retrieve category id
            cat_id = getCatID(cat_name)

            #Insert into the database
            connection = sqlite3.connect('books_rate.db')
            cursor = connection.cursor()

            query = 'INSERT INTO books (title,price,rate,book_catid) VALUES(?,?,?,?)'
            data = [title,price,rating,cat_id]

            cursor.execute(query,data)
            connection.commit()
            connection.close()
        except:
            print('an error occured')
            break

A Light in the Attic in category poetry was inserted successfully!
Tipping the Velvet in category historical fiction was inserted successfully!
Soumission in category fiction was inserted successfully!
Sharp Objects in category mystery was inserted successfully!
Sapiens: A Brief History of Humankind in category history was inserted successfully!
The Requiem Red in category young adult was inserted successfully!
The Dirty Little Secrets of Getting Your Dream Job in category business was inserted successfully!
The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull in category default was inserted successfully!
The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics in category default was inserted successfully!
The Black Maria in category poetry was inserted successfully!
Starving Hearts (Triangular Trade Trilogy, #1) in category default was inserted successfully!
Shakespeare's Sonnets in category poetry was inserted

Immunity: How Elie Metchnikoff Changed the Course of Modern Medicine in category science was inserted successfully!
I Hate Fairyland, Vol. 1: Madly Ever After (I Hate Fairyland (Compilations) #1-5) in category sequential art was inserted successfully!
I am a Hero Omnibus Volume 1 in category sequential art was inserted successfully!
How to Be Miserable: 40 Strategies You Already Use in category self help was inserted successfully!
Her Backup Boyfriend (The Sorensen Family #1) in category romance was inserted successfully!
Giant Days, Vol. 2 (Giant Days #5-8) in category sequential art was inserted successfully!
Forever and Forever: The Courtship of Henry Longfellow and Fanny Appleton in category historical fiction was inserted successfully!
First and First (Five Boroughs #3) in category romance was inserted successfully!
Fifty Shades Darker (Fifty Shades #2) in category romance was inserted successfully!
Everydata: The Misinformation Hidden in the Little Data You Consume Every Day in c

Dark Notes in category erotica was inserted successfully!
Daring Greatly: How the Courage to Be Vulnerable Transforms the Way We Live, Love, Parent, and Lead in category nonfiction was inserted successfully!
Close to You in category christian fiction was inserted successfully!
Chasing Heaven: What Dying Taught Me About Living in category spirituality was inserted successfully!
Big Magic: Creative Living Beyond Fear in category nonfiction was inserted successfully!
Becoming Wise: An Inquiry into the Mystery and Art of Living in category nonfiction was inserted successfully!
Beauty Restored (Riley Family Legacy Novellas #3) in category default was inserted successfully!
Batman: The Long Halloween (Batman) in category sequential art was inserted successfully!
Batman: The Dark Knight Returns (Batman) in category sequential art was inserted successfully!
Ayumi's Violin in category default was inserted successfully!
Anonymous in category default was inserted successfully!
Amy Meets the Saint

Hold Your Breath (Search and Rescue #1) in category romance was inserted successfully!
Hamilton: The Revolution in category nonfiction was inserted successfully!
Greek Mythic History in category default was inserted successfully!
God: The Most Unpleasant Character in All Fiction in category religion was inserted successfully!
Glory over Everything: Beyond The Kitchen House in category historical fiction was inserted successfully!
Feathers: Displays of Brilliant Plumage in category art was inserted successfully!
Far & Away: Places on the Brink of Change: Seven Continents, Twenty-Five Years in category nonfiction was inserted successfully!
Every Last Word in category default was inserted successfully!
Eligible (The Austen Project #4) in category fiction was inserted successfully!
El Deafo in category sequential art was inserted successfully!
Eight Hundred Grapes in category fiction was inserted successfully!
Eaternity: More than 150 Deliciously Easy Vegan Recipes for a Long, Healthy, Sat

Mothering Sunday in category fiction was inserted successfully!
Mother, Can You Not? in category nonfiction was inserted successfully!
M Train in category autobiography was inserted successfully!
Lilac Girls in category historical fiction was inserted successfully!
Lies and Other Acts of Love in category fiction was inserted successfully!
Lab Girl in category autobiography was inserted successfully!
Keep Me Posted in category womens fiction was inserted successfully!
It Didn't Start with You: How Inherited Family Trauma Shapes Who We Are and How to End the Cycle in category psychology was inserted successfully!
Grey (Fifty Shades #4) in category womens fiction was inserted successfully!
Exit, Pursued by a Bear in category young adult was inserted successfully!
Daredevils in category fiction was inserted successfully!
Cravings: Recipes for What You Want to Eat in category food and drink was inserted successfully!
Born for This: How to Find the Work You Were Meant to Do in category busin

The Story of Art in category art was inserted successfully!
The Origin of Species in category science was inserted successfully!
The Great Gatsby in category default was inserted successfully!
The Good Girl in category default was inserted successfully!
The Glass Castle in category add a comment was inserted successfully!
The Faith of Christopher Hitchens: The Restless Soul of the World's Most Notorious Atheist in category biography was inserted successfully!
The Drowning Girls in category add a comment was inserted successfully!
The Constant Princess (The Tudor Court #1) in category historical fiction was inserted successfully!
The Bourne Identity (Jason Bourne #1) in category fiction was inserted successfully!
The Bachelor Girl's Guide to Murder (Herringford and Watts Mysteries #1) in category mystery was inserted successfully!
The Art Book in category art was inserted successfully!
The 7 Habits of Highly Effective People: Powerful Lessons in Personal Change in category default was i

The Dream Thieves (The Raven Cycle #2) in category default was inserted successfully!
The Darkest Corners in category young adult was inserted successfully!
The Crossover in category poetry was inserted successfully!
The 5th Wave (The 5th Wave #1) in category default was inserted successfully!
Tell the Wind and Fire in category fantasy was inserted successfully!
Tell Me Three Things in category young adult was inserted successfully!
Talking to Girls About Duran Duran: One Young Man's Quest for True Love and a Cooler Haircut in category nonfiction was inserted successfully!
Siddhartha in category fiction was inserted successfully!
Shiver (The Wolves of Mercy Falls #1) in category default was inserted successfully!
Remember Me? in category default was inserted successfully!
Red Dragon (Hannibal Lecter #1) in category horror was inserted successfully!
Peak: Secrets from the New Science of Expertise in category science was inserted successfully!
My Mother Was Nuts in category nonfiction wa

Walk the Edge (Thunder Road #2) in category young adult was inserted successfully!
Voyager (Outlander #3) in category historical fiction was inserted successfully!
Very Good Lives: The Fringe Benefits of Failure and the Importance of Imagination in category nonfiction was inserted successfully!
Vegan Vegetarian Omnivore: Dinner for Everyone at the Table in category food and drink was inserted successfully!
Unstuffed: Decluttering Your Home, Mind, and Soul in category nonfiction was inserted successfully!
Under the Banner of Heaven: A Story of Violent Faith in category nonfiction was inserted successfully!
Two Boys Kissing in category young adult was inserted successfully!
Twilight (Twilight #1) in category default was inserted successfully!
Twenties Girl in category womens fiction was inserted successfully!
Trespassing Across America: One Man's Epic, Never-Done-Before (and Sort of Illegal) Hike Across the Heartland in category nonfiction was inserted successfully!
Three-Martini Lunch i

Notes from a Small Island (Notes From a Small Island #1) in category nonfiction was inserted successfully!
Night (The Night Trilogy #1) in category nonfiction was inserted successfully!
Neither Here nor There: Travels in Europe in category travel was inserted successfully!
Naked in category humor was inserted successfully!
Morning Star (Red Rising #3) in category default was inserted successfully!
Miracles from Heaven: A Little Girl, Her Journey to Heaven, and Her Amazing Story of Healing in category nonfiction was inserted successfully!
Midnight Riot (Peter Grant/ Rivers of London - books #1) in category fantasy was inserted successfully!
Me Talk Pretty One Day in category autobiography was inserted successfully!
Manuscript Found in Accra in category add a comment was inserted successfully!
Lust & Wonder in category autobiography was inserted successfully!
Lila (Gilead #3) in category fiction was inserted successfully!
Life, the Universe and Everything (Hitchhiker's Guide to the Galax

Fruits Basket, Vol. 2 (Fruits Basket #2) in category sequential art was inserted successfully!
Diary of a Minecraft Zombie Book 1: A Scare of a Dare (An Unofficial Minecraft Book) in category childrens was inserted successfully!
Y: The Last Man, Vol. 1: Unmanned (Y: The Last Man #1) in category sequential art was inserted successfully!
While You Were Mine in category historical fiction was inserted successfully!
Where Lightning Strikes (Bleeding Stars #3) in category romance was inserted successfully!
When I'm Gone in category fiction was inserted successfully!
Ways of Seeing in category art was inserted successfully!
Vampire Knight, Vol. 1 (Vampire Knight #1) in category paranormal was inserted successfully!
Vampire Girl (Vampire Girl #1) in category fantasy was inserted successfully!
Twenty Love Poems and a Song of Despair in category poetry was inserted successfully!
Travels with Charley: In Search of America in category nonfiction was inserted successfully!
Three Wishes (River of T

In [21]:
#Test that insertion was done successfully
connection = sqlite3.connect('books_rate.db')
cursor = connection.cursor()
cursor.execute('SELECT * FROM books')
rows = cursor.fetchall()
connection.close()

rows

[('A Light in the Attic', 51.77, 3, 21),
 ('Tipping the Velvet', 53.74, 1, 2),
 ('Soumission', 50.1, 1, 8),
 ('Sharp Objects', 47.82, 4, 1),
 ('Sapiens: A Brief History of Humankind', 54.23, 5, 30),
 ('The Requiem Red', 22.65, 1, 19),
 ('The Dirty Little Secrets of Getting Your Dream Job', 33.34, 4, 33),
 ('The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull',
  17.93,
  3,
  13),
 ('The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics',
  22.6,
  4,
  13),
 ('The Black Maria', 52.15, 1, 21),
 ('Starving Hearts (Triangular Trade Trilogy, #1)', 13.99, 2, 13),
 ("Shakespeare's Sonnets", 20.66, 4, 21),
 ('Set Me Free', 17.46, 5, 19),
 ("Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)", 52.29, 5, 3),
 ('Rip it Up and Start Again', 35.02, 5, 12),
 ('Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991',
  57.25,
  3,
  12),
 ('Olio', 23.88, 1, 21),
 ('Mesaerion: The Best Scien

In [23]:
#Test that insertion was done successfully
connection = sqlite3.connect('books_rate.db')
cursor = connection.cursor()
cursor.execute('SELECT * FROM categories')
rows = cursor.fetchall()
connection.close()

rows

[(0, 'travel'),
 (1, 'mystery'),
 (2, 'historical fiction'),
 (3, 'sequential art'),
 (4, 'classics'),
 (5, 'philosophy'),
 (6, 'romance'),
 (7, 'womens fiction'),
 (8, 'fiction'),
 (9, 'childrens'),
 (10, 'religion'),
 (11, 'nonfiction'),
 (12, 'music'),
 (13, 'default'),
 (14, 'science fiction'),
 (15, 'sports and games'),
 (16, 'add a comment'),
 (17, 'fantasy'),
 (18, 'new adult'),
 (19, 'young adult'),
 (20, 'science'),
 (21, 'poetry'),
 (22, 'paranormal'),
 (23, 'art'),
 (24, 'psychology'),
 (25, 'autobiography'),
 (26, 'parenting'),
 (27, 'adult fiction'),
 (28, 'humor'),
 (29, 'horror'),
 (30, 'history'),
 (31, 'food and drink'),
 (32, 'christian fiction'),
 (33, 'business'),
 (34, 'biography'),
 (35, 'thriller'),
 (36, 'contemporary'),
 (37, 'spirituality'),
 (38, 'academic'),
 (39, 'self help'),
 (40, 'historical'),
 (41, 'christian'),
 (42, 'suspense'),
 (43, 'short stories'),
 (44, 'novels'),
 (45, 'health'),
 (46, 'politics'),
 (47, 'cultural'),
 (48, 'erotica'),
 (49, 'cr

In [22]:
getCatID('poetry')

21